# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
import torch

import time

In [2]:
from a2pm import A2PMethod
from a2pm.callbacks import BaseCallback, MetricCallback, TimeCallback
from a2pm.patterns import BasePattern, CombinationPattern, IntervalPattern
from a2pm.wrappers import BaseWrapper, KerasWrapper, SklearnWrapper, TorchWrapper

In [3]:
X_train = np.load('X-IIoT-pre-processed/x_train.npy')
y_train = np.load('X-IIoT-pre-processed/y_train.npy')
X_val = np.load('X-IIoT-pre-processed/x_val.npy')
y_val = np.load('X-IIoT-pre-processed/y_val.npy')
X_test = np.load('X-IIoT-pre-processed/x_test.npy')
y_test = np.load('X-IIoT-pre-processed/y_test.npy')

# Train

We train the baseline model to get an impression of how our model perform on an IDS dataset without any adversarial samples.

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [6]:
model_rf = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 50, 100],
}

grid_search_rf = GridSearchCV(model_rf, param_grid, cv=3, scoring='accuracy', verbose=3)
grid_search_rf.fit(X_train, y_train)

print("Best parameters found: ", grid_search_rf.best_params_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3] END ...max_depth=None, n_estimators=10;, score=0.995 total time=   5.1s
[CV 2/3] END ...max_depth=None, n_estimators=10;, score=0.995 total time=   4.8s
[CV 3/3] END ...max_depth=None, n_estimators=10;, score=0.995 total time=   5.2s
[CV 1/3] END ...max_depth=None, n_estimators=50;, score=0.995 total time=  24.8s
[CV 2/3] END ...max_depth=None, n_estimators=50;, score=0.996 total time=  25.1s
[CV 3/3] END ...max_depth=None, n_estimators=50;, score=0.995 total time=  25.4s
[CV 1/3] END ..max_depth=None, n_estimators=100;, score=0.996 total time=  51.0s
[CV 2/3] END ..max_depth=None, n_estimators=100;, score=0.996 total time=  51.8s
[CV 3/3] END ..max_depth=None, n_estimators=100;, score=0.995 total time=  51.6s
[CV 1/3] END .....max_depth=50, n_estimators=10;, score=0.995 total time=   5.2s
[CV 2/3] END .....max_depth=50, n_estimators=10;, score=0.995 total time=   5.1s
[CV 3/3] END .....max_depth=50, n_estimators=10;,

# Evaluate

Evaluate the RF model on the IDS dataset, once again with no adversarial samples.

## Helpers

In [7]:
def metrics_master(model, X_test, y_test):
    metrics_weighted(model, X_test, y_test)
    metrics_macro(model, X_test, y_test)

def metrics_macro(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Macro Precision: {precision * 100:.2f}%")
    print(f"Macro Recall: {recall * 100:.2f}%")
    print(f"Macro F1 Score: {f1 * 100:.2f}%")
    
def metrics_weighted(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Weighted Precision: {precision * 100:.2f}%")
    print(f"Weighted Recall: {recall * 100:.2f}%")
    print(f"Weighted F1 Score: {f1 * 100:.2f}%")

## Scores

In [8]:
rf_model = grid_search_rf.best_estimator_
metrics_master(rf_model, X_test, y_test)

Accuracy: 99.59%
Weighted Precision: 99.59%
Weighted Recall: 99.59%
Weighted F1 Score: 99.58%
Accuracy: 99.59%
Macro Precision: 99.40%
Macro Recall: 91.58%
Macro F1 Score: 94.51%


# A2PM Generation

Generate A2PM adversarial samples for further experiments

Since we know the data is pre-processed, the categorical variables will be binary columns, and conversely the numerical variables will be the others. Let's find these columns.

In [9]:
def find_binary_columns(X_train):
    binary_columns = []
    for col in range(X_train.shape[1]):
        unique_values = np.unique(X_train[:, col])
        if set(unique_values).issubset({0, 1}):
            binary_columns.append(col)
    return binary_columns

binary_columns = find_binary_columns(X_train)

numerical_columns = []
for i in range(0,58):
    if i not in binary_columns:
        numerical_columns.append(i)

The following configuration is the boilerplate found on the A2PM github (https://github.com/vitorinojoao/a2pm). The only modifications are that the interval pattern is applied to the numerical columns, and the combination pattern is applied to the categorical (binary) variables. I removed integer features since the numerical columns are float values, and removed locked_features as I wasn't sure which features needed to be kept static (data was given as processed .npy files)

In [10]:
RF_base = grid_search_rf.best_estimator_

classifier = SklearnWrapper(RF_base)


# rule of thumb: Interval for numerical, combination for categorical
pattern = (
    {
        "type": "interval",
        "features": numerical_columns,
        #"integer_features": list(range(10, 20)),
        "ratio": 0.1,
        "max_ratio": 0.3,
        "missing_value": 0.0,
        "probability": 0.6,
    },
    {
        "type": "combination",
        "features": binary_columns,
        #"locked_features": list(range(30, 40)), # Locks some features to ensure validity. Not using this because data is .npy and unreadable
        "probability": 0.4,
    },
)

method = A2PMethod(pattern)

We will generate attacks based off the entire dataset. We will resplit the train-test sets afterward in our experiments

In [11]:
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

In [13]:
start_time = time.time()

X_adversarial = method.fit_generate(classifier, X, y)

training_time = time.time() - start_time
print(f"Attack Time: {training_time}")

Attack Time: 119.56253099441528


# Scores after A2PM Attack

Here we are evaluating how our trained (unprotected) RF model performs on the full A2PM dataset. Each sample is now perturbed, but should still correspond to the original target classes y

In [14]:
metrics_master(rf_model, X_adversarial, y)

/Users/elvin/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 55.22%
Weighted Precision: 45.97%
Weighted Recall: 55.22%
Weighted F1 Score: 43.16%
Accuracy: 55.22%
Macro Precision: 19.09%
Macro Recall: 8.17%
Macro F1 Score: 7.61%


/Users/elvin/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Adversarial Training Preprocess

As per many adversarial defense papers, adversarial training has been cited as one of the best ways to improve resilience. We will retrain our model with the original data, in addition to an added 10% of the data being adversarial samples with an "adversarial" class (class 19).

In [15]:
adv_samples_num = int(588965*0.10/0.9) 
adv_samples_num # 10%

65440

In [31]:
X_adversarial_indices = np.random.choice(X_adversarial.shape[0], size=adv_samples_num, replace=False)
X_adversarial_sampled = X_adversarial[X_adversarial_indices]

y_adv = np.full(adv_samples_num, 19)
y_adv

array([19, 19, 19, ..., 19, 19, 19])

In [17]:
X_new = X.copy()
y_new = y.copy()

X_combined = np.vstack((X_new, X_adversarial_sampled))
y_combined = np.concatenate((y_new, y_adv))

# Adversarial Training

Our whole data now contains 10% A2PM samples. Assuming a uniformly random 80/20 split, our training data will be expected to contain 8% A2PM samples, while the testing data is expected to contain 2% A2PM samples. 

In [18]:
X_train_adv, X_test_adv, y_train_adv, y_test_adv = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

In [23]:
model_rf_protected = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 50, 100],
}

grid_search_rf_protected = GridSearchCV(model_rf_protected, param_grid, cv=3, scoring='accuracy', verbose=3)
grid_search_rf_protected.fit(X_train_adv, y_train_adv)

print("Best parameters found: ", grid_search_rf.best_params_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3] END ...max_depth=None, n_estimators=10;, score=0.995 total time=   7.7s
[CV 2/3] END ...max_depth=None, n_estimators=10;, score=0.995 total time=   7.3s
[CV 3/3] END ...max_depth=None, n_estimators=10;, score=0.995 total time=   7.5s
[CV 1/3] END ...max_depth=None, n_estimators=50;, score=0.996 total time=  38.4s
[CV 2/3] END ...max_depth=None, n_estimators=50;, score=0.996 total time=  38.5s
[CV 3/3] END ...max_depth=None, n_estimators=50;, score=0.996 total time=  39.9s
[CV 1/3] END ..max_depth=None, n_estimators=100;, score=0.996 total time= 1.3min
[CV 2/3] END ..max_depth=None, n_estimators=100;, score=0.996 total time= 1.3min
[CV 3/3] END ..max_depth=None, n_estimators=100;, score=0.996 total time= 1.3min
[CV 1/3] END .....max_depth=50, n_estimators=10;, score=0.995 total time=   7.5s
[CV 2/3] END .....max_depth=50, n_estimators=10;, score=0.995 total time=   7.8s
[CV 3/3] END .....max_depth=50, n_estimators=10;,

In [43]:
rf_model_protected = grid_search_rf_protected.best_estimator_

# Scores (Binary A2PM Detection after Adversarial Training)

We want to see how well our protected model can detect A2PM samples. We run it against the unseen A2PM set

In [45]:
X_adversarial_unseen = X_adversarial[np.logical_not(np.isin(np.arange(X_adversarial.shape[0]), X_adversarial_indices))]
len(X_adversarial_unseen)

523525

In [46]:
protected_adversarial_preds = rf_model_protected.predict(X_adversarial_unseen)

All we need to know is if the model correctly predicts every sample as class 19 (A2PM), and as such we will treat this as a binary classification problem.

In [47]:
y_adv_full = np.full(len(X_adversarial_unseen), 19)

positive_class = 19
y_test_binary = (y_adv_full == positive_class).astype(int)
adversarial_preds_binary = (protected_adversarial_preds == positive_class).astype(int)

In [48]:
adversarial_accuracy = accuracy_score(y_test_binary, adversarial_preds_binary)
adversarial_precision = precision_score(y_test_binary, adversarial_preds_binary, average='binary')
adversarial_recall = recall_score(y_test_binary, adversarial_preds_binary, average='binary')
adversarial_f1 = f1_score(y_test_binary, adversarial_preds_binary, average='binary')

print(f"Adversarial Accuracy: {adversarial_accuracy * 100:.2f}%")
print(f"Adversarial Precision: {adversarial_precision * 100:.2f}%")
print(f"Adversarial Recall: {adversarial_recall * 100:.2f}%")
print(f"Adversarial F1 Score: {adversarial_f1 * 100:.2f}%")

Adversarial Accuracy: 99.95%
Adversarial Precision: 100.00%
Adversarial Recall: 99.95%
Adversarial F1 Score: 99.98%


# Scores (10% A2PM Samples, no Adv Training)

Here we benchmark how well our unprotected model performs on a test set with 10% A2PM samples and 90% real samples

In [50]:
metrics_master(rf_model, X_test_adv, y_test_adv)

/Users/elvin/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 89.89%
Weighted Precision: 81.32%
Weighted Recall: 89.89%
Weighted F1 Score: 85.27%
Accuracy: 89.89%
Macro Precision: 91.29%
Macro Recall: 94.22%
Macro F1 Score: 92.64%


/Users/elvin/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Scores (10% A2PM Samples w/ Adv Training)

Here we benchmark how well our protected model performs on a test set with 10% A2PM samples and 90% real samples

In [51]:
metrics_master(rf_model_protected, X_test_adv, y_test_adv)

Accuracy: 99.64%
Weighted Precision: 99.64%
Weighted Recall: 99.64%
Weighted F1 Score: 99.63%
Accuracy: 99.64%
Macro Precision: 99.50%
Macro Recall: 94.01%
Macro F1 Score: 96.30%
